In [1]:
import tensorflow as tf
import numpy as np
import gensim
import sys
import datetime,time
from six.moves import cPickle as pickle

In [2]:
def log_time_model(s):
    now = datetime.datetime.now()
    print "[%s][%s.%s][%s]" %(time.strftime('%Y-%m-%d %H-%M-%S', time.localtime(time.time())), now.second, now.    microsecond, s)

In [3]:
def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, WordVec_Length, Word_Length_Default * 2,num_channels)).astype(np.float32)
    
    return dataset

In [4]:
def reAdd(dataset,Filter_Width):
    # give pairs's distance,then distance = Filter_Length * File_Width 
    # and  return real input data x  =   [None, WordVec_Length, Word_Length_Default * 2 + Filter_Length,num_channels]
    b = np.zeros([Filter_Length,Filter_Width,num_channels])
    return np.insert(dataset, Word_Length_Default, values=b, axis=1)
    

In [5]:
# define wordvec parameter 
WordVec_Length = 50
WordVec_Depth = 1

# define batch size 
BatchSize = 2

# define filter parameter
Filter_Length = WordVec_Length
Filter_Width = 3
num_channels = WordVec_Depth
### define filter depth, then equal next layer filter_curmat_depth
Filter_Depth = 9

# one question 's max word length , is equal with lenth of sentence.
Word_Length_Default = 50

x = tf.placeholder(tf.float32,[BatchSize, WordVec_Length, Word_Length_Default * 2 + Filter_Width,num_channels],name='x-input1')
#  dimension[0] is batch size,dim[1] and dim[2] means pairs's size,dim[3] is depth of pairs

log_time_model("x.shape()" + str(x.get_shape()))


[2017-04-21 19-15-57][57.986780][x.shape()TensorShape([Dimension(2), Dimension(50), Dimension(103), Dimension(1)])]


In [6]:
#def inference(input_tensor):
with tf.variable_scope('layer1-conv1'):
    
    conv1_filter_weights = tf.get_variable('weight',
                                          [Filter_Length,Filter_Width,num_channels,Filter_Depth],
                                          initializer=tf.truncated_normal_initializer(stddev=0.1))
    
    log_time_model("conv1_filter_weights = "+str(conv1_filter_weights.get_shape()))

    biases1 = tf.get_variable('biases',[Filter_Depth],initializer=tf.constant_initializer(0.1))
    
    log_time_model("biases1 = "+str(biases1.get_shape()))
    
    simM = tf.get_variable('M',[Filter_Depth,Filter_Depth],initializer=tf.truncated_normal_initializer(stddev=0.1))
    # 定义相似度矩阵 simM
    
    log_time_model("simM = "+str(simM.get_shape()))
    
    conv1 = tf.nn.conv2d(x, conv1_filter_weights, strides = [1,1,1,1],padding='SAME')
    
    log_time_model("conv1 = "+str(conv1.get_shape()))

    bias1 = tf.nn.bias_add(conv1, biases1)
    
    log_time_model("bias1 = "+str(bias1.get_shape()))

    actived_conv1 = tf.nn.relu(bias1)
    
    log_time_model("actived_conv1 = "+str(actived_conv1.get_shape()))
    
    
    Conv_Length = 2
#     if actived_conv1.get_shape()[2].value == Filter_Depth:
#         Conv_Length = actived_conv1.get_shape()[3].value
        
    pool = tf.nn.max_pool(actived_conv1,ksize=[1,1,Conv_Length/2,1],strides=[1,Conv_Length/2,Conv_Length/2,1],padding='VALID')
    
    log_time_model("pool.get_shape() = "+ str(pool.get_shape()))
    
    
    
    # NEXT STEP: MAXPOOL 定义 两个不需要训练的变量，maxpool ()
    # 定义一个需要训练的矩阵M
    # 两个变量矩阵相乘
    # 拉直拼接
    # 全连接
    # softmax
    

    
        

[2017-04-21 19-15-58][58.58618][conv1_filter_weights = TensorShape([Dimension(51), Dimension(3), Dimension(1), Dimension(9)])]
[2017-04-21 19-15-58][58.60744][biases1 = TensorShape([Dimension(9)])]
[2017-04-21 19-15-58][58.65574][simM = TensorShape([Dimension(9), Dimension(9)])]


ValueError: ('filter must not be larger than the input: ', 'Filter: [', Dimension(51), 'x', Dimension(3), '] ', 'Input: [', Dimension(50), 'x', Dimension(103), '] ')

In [ ]:
# with tf.Session() as sess:
#     print sess.run(a.initializer)
#     print dir(a.get_shape()[1].value)
#     print a.get_shape()[1].value

In [ ]:
# read dataset
def openFile():
    pickle_file = 'dataset.pickle'
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)
        ##第一个句子的样本矩阵为np.ndarray((samplesize, 300, 50), dtype=np.float32))
        train_dataset1 = data['train_dataset1'] 
        ##第二个句子的样本矩阵为np.ndarray((samplesize, 300, 50), dtype=np.float32))
        train_dataset2 = data['train_dataset2']
        ##类别标识np.ndarray((samplesize, dtype = np.int32))
        train_labels = data['train_labels']
        ##验证集
        valid_dataset1 = data['valid_dataset1'] 
        valid_dataset2 = data['valid_dataset2']
        valid_labels = data['valid_labels']
        ##测试集
        test_dataset1 = data['test_dataset1'] 
        test_dataset2 = data['test_dataset2']